# Prompt templates

We can use Python built-in templates with substitutions, but it makes it difficult to substitute in real-time:

In [14]:
prompt_template = (
    "Given a job description, decide whether it suites a junior Java developer."
    "\nJOB DESCRIPTION:\n{job_description}\n"
)
result = llm.invoke(prompt_template.format(job_description=job_description))
print(result)

content='Okay, based on the job description "test job description", it\'s impossible to definitively say whether it suits a junior Java developer.\n\n**Why?**\n\n*   **Lack of Information:** A "test job description" provides absolutely no details about the required skills, experience, responsibilities, or technologies involved.\n\n**To determine suitability, we would need a real job description that includes details such as:**\n\n*   **Required skills:** Java, Spring, SQL, REST APIs, etc.\n*   **Experience level:** Entry-level, 0-2 years, etc.\n*   **Responsibilities:** Writing code, testing, debugging, participating in code reviews, etc.\n*   **Technologies used:** Specific frameworks, libraries, databases, etc.\n*   **Team environment:** Agile, Scrum, etc.\n\nIf the description mentions basic Java concepts, some familiarity with common frameworks, and a willingness to learn, it\'s *more likely* suitable for a junior developer. If it\'s full of advanced concepts and expects independen

Instead, we can use built-in templates provided by LangChain:

In [15]:
from langchain_core.prompts import PromptTemplate

lc_prompt_template = PromptTemplate.from_template(prompt_template)
lc_prompt_template.invoke({"job_description": "fake_jd"})

StringPromptValue(text='Given a job description, decide whether it suites a junior Java developer.\nJOB DESCRIPTION:\nfake_jd\n')

In [16]:
from langchain_core.output_parsers import StrOutputParser

lc_prompt_template = PromptTemplate.from_template(prompt_template)
chain = lc_prompt_template | llm | StrOutputParser()
chain.invoke({"job_description": job_description})

'Okay, I\'ve reviewed the job description: "test job description".\n\nBased on that incredibly limited information, **yes, it *could* suit a junior Java developer, but it\'s impossible to say for sure.**\n\nHere\'s why:\n\n*   **Lack of Detail:**  A junior Java developer usually needs a job description that specifies the technologies, frameworks, and types of projects they will be working on.  "test job description" doesn\'t provide any of that.\n*   **Possibility of Entry-Level Testing:** It could be a job for testing Java applications, which a junior developer could potentially do (especially with guidance).\n*   **Could Be Anything:** It\'s so vague it could literally be anything. Maybe it\'s a placeholder, or maybe it\'s deliberately sparse.\n\n**To make a real decision, you\'d need a *real* job description. Look for keywords like:**\n\n*   **Required Skills:** Java, Spring, Spring Boot, REST APIs, SQL, Git, Maven/Gradle, Unit Testing (JUnit/Mockito), etc.\n*   **Responsibilities:*

In [17]:
chain.invoke(job_description)

'Okay, I\'ve reviewed the job description: "test job description".\n\nBased on this *extremely* limited information, it\'s impossible to definitively say whether it suits a junior Java developer.  However, the fact that it\'s labeled "test" suggests it likely lacks the specific requirements and responsibilities needed for a real job.\n\nHere\'s a breakdown of why and what information is needed to make a proper determination:\n\n**Why it\'s impossible to say with current information:**\n\n*   **No Technologies Mentioned:**  It doesn\'t specify Java or any related technologies (Spring, Hibernate, etc.).\n*   **No Experience Level Indicated:** It doesn\'t mention experience requirements.\n*   **No Responsibilities Listed:**  We have no idea what the job entails.\n*   **No Skills Required:**  We don\'t know what skills are needed to perform the job.\n\n**What Information is needed to assess suitability for a Junior Java Developer:**\n\nTo determine if the job suits a Junior Java Developer,

We can also use a special placeholder for messages:

In [18]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import SystemMessagePromptTemplate


msg_template = HumanMessagePromptTemplate.from_template(prompt_template)
msg_example = msg_template.format(job_description="fake_jd")

chat_prompt_template = ChatPromptTemplate.from_messages([SystemMessage(content="You are a helpful assistant."), msg_template])
chain = chat_prompt_template | llm | StrOutputParser()
chain.invoke({"job_description": job_description})

'Okay, I\'ve reviewed the job description "test job description".\n\n**Verdict:**\n\nThis job description is too vague to determine if it suits a junior Java developer.\n\n**Reasoning:**\n\nA junior developer role typically requires some indication of the following:\n\n*   **Required Skills:** Does it mention Java? Does it mention specific Java frameworks (like Spring, Spring Boot, Hibernate)? Does it mention related technologies like databases (SQL, NoSQL), or build tools (Maven, Gradle)?\n*   **Experience Level:** Does it explicitly state "Junior," "Entry-Level," or indicate a number of years of experience (e.g., 0-2 years)?\n*   **Responsibilities:** Does it describe tasks that a junior developer would typically handle, such as:\n    *   Writing code under the supervision of senior developers.\n    *   Debugging and testing existing code.\n    *   Implementing well-defined features.\n    *   Participating in code reviews.\n    *   Learning new technologies.\n*   **Teamwork and Commu

In [19]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."),
     ("human", prompt_template)])
chain = chat_prompt_template | llm | StrOutputParser()
chain.invoke({"job_description": job_description})

'Okay, I\'ve reviewed the job description "test job description".\n\nWithout any details about the required skills, experience, or responsibilities, it\'s impossible to definitively say whether it suits a junior Java developer. \n\n**To determine suitability, I would need more information. Ideally, a good job description for a Junior Java Developer role would include the following:**\n\n*   **Required Skills:**\n    *   **Java:** Core Java knowledge (data structures, algorithms, OOP principles).\n    *   **IDE:** Experience with IDEs like IntelliJ IDEA or Eclipse.\n    *   **Build Tools:** Familiarity with Maven or Gradle.\n    *   **Testing:** Basic understanding of unit testing with frameworks like JUnit or TestNG.\n    *   **Version Control:** Experience with Git.\n    *   **Databases:** Basic knowledge of SQL and relational databases (e.g., MySQL, PostgreSQL).\n    *   **Web Technologies (Optional):** Exposure to Servlets, JSP, or basic web frameworks is a plus.\n*   **Responsibili

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."),
     ("placeholder", "{history}"),
     # same as MessagesPlaceholder("history"),
     ("human", prompt_template)])

In [21]:
chat_prompt_template.invoke("fake")

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Given a job description, decide whether it suites a junior Java developer.\nJOB DESCRIPTION:\nfake\n', additional_kwargs={}, response_metadata={})])

In [22]:
len(chat_prompt_template.invoke({"job_description": "fake", "history": [("human", "hi!"), ("ai", "hi!")]}).messages)

4

In [23]:
examples = [
    {"question": "q1", "answer": "a1"},
    {"question": "q2", "answer": "a2"}
]
test_template = PromptTemplate.from_template("substituted a: {a}")

And we can use partial substitutions with templates:

In [24]:
system_template = PromptTemplate.from_template("a: {a} b: {b}")
system_template_part = system_template.partial(
    a="a" # you also can provide a function here
)
print(system_template_part.invoke({"b": "b"}).text)

system_template_part.invoke({"b": "b"}).text == system_template_part.format(b="b")

a: a b: b


True

In [25]:
system_template_part1 = PromptTemplate.from_template("a: {a}")
system_template_part2 = PromptTemplate.from_template("b: {b}")
system_template = system_template_part1 + system_template_part2
print(system_template_part.invoke({"a": "a", "b": "b"}).text)

a: a b: b


In [26]:
system_prompt_template = PromptTemplate.from_template("a: {a} b: {b}")
chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_prompt_template.template),
     ("human", "hi"),
     ("ai", "{c}")])

messages = chat_prompt_template.invoke({"a": "a", "b": "b", "c": "c"}).messages
print(len(messages))
print(messages[0].content)

3
a: a b: b
